# Quantifications of area and growth

## Documentation and Imports


Created on 01-04-2021

Author: Valentin Laplaud

This code is loading data from different experiment, created by ShapeQuatification. It plots those various data and compare them to each other.

In [ ]:
## Clean up before script start 

for element in dir():
    if element[0:2] != "__":
        del globals()[element]

import gc
gc.collect()


print('\033[1m' + '\033[4m' + '\nRunning :  ''ExperimentComparisonNB''\n' + '\033[0m')

# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2
mpl.rcParams['axes.facecolor'] = COLOR2

import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import seaborn as sns
from cycler import cycler
#Default colors
colorcycle = [plt.get_cmap('gist_rainbow')(1. * i/30) for i in range(30)]
mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)

# numbers handling
import numpy as np
import numpy.matlib as mtl
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.stats import ranksums, linregress

# images handling
from skimage import io
from skimage.filters import threshold_otsu, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv

# to hide known warnings
import warnings

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../')
import VallapFunc as vf


## Define functions

### Plot gemmae area in time

In [ ]:
def compareAreas(GDs, Labels, colors,P, Title, **kwargs):

    showcurve = False
    showbox = False
    AllSigs = True
    IndividualPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'showcurve':
            showcurve = value 
        elif key == 'showbox':
            showbox = value
        elif key == 'sigpairs':
            sigpairs = value
            AllSigs = False
        elif key == 'indiplots' :
            IndividualPlots = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    # Colors
    colorcycle = [plt.get_cmap('gist_rainbow')(1. * i/len(GDs)) for i in range(len(GDs))]
    mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)
         
    # check existence of figure folder, if absent, create it
    if not os.path.exists(P + '\\Figures\\AreaGrowth'):
            os.mkdir(P + '\\Figures\\AreaGrowth') # create folder
    
    ######### Curves of evolution ##########
    
    fig2,ax2 = plt.subplots(dpi = 250,facecolor='black')
    fig2.suptitle(Title + ' - Area vs. time')
    plt.xlabel('Time (min)')
    plt.ylabel('Area (mm²)')
    
    fig3,ax3 = plt.subplots(dpi = 250,facecolor='black')
    fig3.suptitle(Title + ' - Norm Area vs. time')
    plt.xlabel('Time (min)')
    plt.ylabel('Area (normalized)')
    
    """  fig10,ax10 = plt.subplots(dpi = 250,facecolor='black')
    fig10.suptitle(Title + ' - Area vs. aligned time')
    plt.xlabel('Time (min)')
    plt.ylabel('Area (mm²)')
    
    fig11,ax11 = plt.subplots(dpi = 250,facecolor='black')
    fig11.suptitle(Title + ' - Norm Area vs. aligned time')
    plt.xlabel('Time (min)')
    plt.ylabel('Area (normalized)')"""
    
    for GD,lab,i in zip(GDs,Labels,range(len(GDs))):
        
        StackList = np.unique(GD.index)
        
        if IndividualPlots:
            imgs = range(49)
            fig1,ax1 = plt.subplots(dpi = 250,facecolor='black')
            fig1.suptitle(lab + ' - Area vs. time')
            plt.xlabel('Time (min)')
            plt.ylabel('Area (mm²)')
            for s in StackList:
                ax1.plot(np.multiply(imgs,30),GD.loc[(GD.index == s) & (GD['Img'].values<49),'Area'],label=s)
            plt.legend(prop={'size': 5})

        # number of ppgs and label
        nPPG = len(GD.loc[GD['Img'] == 0])
        lab = lab + ' - n = ' + str(nPPG)
        
        # Computing mean area over all gemmae for each image
        MeanA = np.empty(49)
        MeanTime = np.empty(49)
        StdA = np.empty(49)

        for im in range(49):

            MeanA[im] = GD.loc[GD['Img'] == im,'Area'].to_numpy().mean()
            MeanTime[im] = im*30
            StdA[im] = GD.loc[GD['Img'] == im,'Area'].to_numpy().std()
        
        nppg = len(GD.loc[GD['Img'] == 0,'Area'].to_numpy())

        ax2.errorbar(MeanTime,MeanA,yerr=StdA/np.sqrt(nppg), capsize=3,label=lab,color = colors[i])
        ax3.errorbar(MeanTime,MeanA/MeanA[0],yerr=StdA/MeanA[0]/np.sqrt(nppg), capsize=3,label=lab,color = colors[i])
        
        """# Computing mean of area with alignement in time based on fit
        
        AlAreas = np.empty((49,len(StackList)))
        AlAreas[:] = np.nan
        
        maxshift = 0
        
        for ii,s in zip(range(len(StackList)),StackList):
            ImgShift = GD.loc[(GD.index == s) & (GD['Img'].values == 0), 'tdebShift'].values   
            maxshift = int(np.max([maxshift,ImgShift]))
            
            for im in range(49):
                
                if (im+ImgShift)<np.max(GD['Img']+1):

                    AlAreas[im,ii] = GD.loc[(GD.index == s) & (GD['Img'].values == im+ImgShift), 'Area'].values
        
        print(np.max(GD['Img']+1)-maxshift)
        AlMeanA = np.nanmean(AlAreas[range(np.max(GD['Img']+1)-maxshift)], axis=1)
        AlStdA = np.nanstd(AlAreas[range(np.max(GD['Img']+1)-maxshift)], axis = 1)

        
        ax10.errorbar(MeanTime[range(np.max(GD['Img']+1)-maxshift)],AlMeanA,yerr=AlStdA/np.sqrt(nppg), capsize=3,label=lab,color = colors[i])
        ax11.errorbar(MeanTime[range(np.max(GD['Img']+1)-maxshift)],AlMeanA/AlMeanA[0],yerr=AlStdA/AlMeanA[0]/np.sqrt(nppg), capsize=3,label=lab,color = colors[i])
      """
    plt.figure(fig2.number)
    plt.legend(prop={'size': 8})
    fig2.savefig(P + '\\Figures\\AreaGrowth\\' + Title + '_AreaCurve.png')
    if not showcurve:
        plt.close(fig2)

    plt.figure(fig3.number)
    plt.legend(prop={'size': 8})
    fig3.savefig(P + '\\Figures\\AreaGrowth\\' + Title + '_NormAreaCurve.png')
    if not showcurve:
        plt.close(fig3)

    """plt.figure(fig10.number)
    plt.legend(prop={'size': 8})
    fig10.savefig(P + '\\Figures\\AliAreaTime_' + Title + '.png')
    if not showcurve:
        plt.close(fig1)

    plt.figure(fig11.number)
    plt.legend(prop={'size': 8})
    fig11.savefig(P + '\\Figures\\AliNormAreaTime_' + Title + '.png')
    if not showcurve:
        plt.close(fig1)"""

    ######### Parameters of fit ###########
    
      
    fig4,ax4 = plt.subplots(dpi = 250,facecolor='black')
    fig4.suptitle(Title + ' - Growth start time')
    plt.ylabel('T start (min)')
      
    fig5,ax5 = plt.subplots(dpi = 250,facecolor='black')
    fig5.suptitle(Title + ' - Growth caracteristic time')
    plt.ylabel('Tau growth (hours)')
      
    fig6,ax6 = plt.subplots(dpi = 250,facecolor='black')
    fig6.suptitle(Title + ' - Starting area')
    plt.ylabel('Starting area from fit (mm²)')
    
    tdebs= [None]*len(GDs)
    taus= [None]*len(GDs)
    stdtdebs= [None]*len(GDs)
    stdtaus= [None]*len(GDs)
    captdeb= [None]*len(GDs)
    captau= [None]*len(GDs)
    medtdeb= [None]*len(GDs)
    medtau= [None]*len(GDs)    
    
    Area0 = [None]*len(GDs)
    stdArea0 = [None]*len(GDs)
    capArea0 = [None]*len(GDs)
    medArea0 = [None]*len(GDs)
    
    for GD,lab,i in zip(GDs,Labels,range(len(GDs))):
        
        # number of ppgs and label
        nPPG = len(GD.loc[GD['Img'] == 0])
        lab = lab + '\nn = ' + str(nPPG)
        
        # Retrieve data
        tdebs[i] = GD.loc[GD['Img'] == 0, 'tdeb']
        taus[i] = GD.loc[GD['Img'] == 0, 'Tau']/60
        stdtdebs[i] = GD.loc[GD['Img'] == 0, 'STDtdeb']
        stdtaus[i] = GD.loc[GD['Img'] == 0, 'STDTau']/60        
        Area0[i] = GD.loc[GD['Img'] == 0, 'A0fit']


        plotprops = {'color':colors[i]}
        flierprops = dict(marker='d', markerfacecolor=colors[i], markersize=8,
                  markeredgecolor='none')
        
        bp4 = ax4.boxplot(tdebs[i], positions = [i], labels = [lab],boxprops=plotprops, capprops =plotprops,
                    flierprops=flierprops,
                    whiskerprops=plotprops,medianprops =plotprops)
        bp5 = ax5.boxplot(taus[i], positions = [i], labels = [lab], boxprops=plotprops, capprops =plotprops,
                    flierprops=flierprops,
                    whiskerprops=plotprops,medianprops =plotprops)
        bp6 = ax6.boxplot(Area0[i], positions = [i], labels = [lab], boxprops=plotprops, capprops =plotprops,
                    flierprops=flierprops,
                    whiskerprops=plotprops,medianprops =plotprops)
    
        captdeb[i] = bp4['caps'][1].get_ydata(orig=True)[0]
        captau[i] = bp5['caps'][1].get_ydata(orig=True)[0]
        capArea0[i] = bp6['caps'][1].get_ydata(orig=True)[0]
        medtdeb[i] = bp4['medians'][0].get_ydata(orig=True)[0]
        medtau[i] = bp5['medians'][0].get_ydata(orig=True)[0]
        medArea0[i] = bp6['medians'][0].get_ydata(orig=True)[0]

    
    steptdeb = np.max(captdeb)*0.125
    steptau = np.max(captau)*0.125
    stepArea0 = np.max(capArea0)*0.125
    
    fullsteptdeb = 0
    fullsteptau = 0
    fullstepArea0 = 0
    
    hmaxtdeb = np.max(captdeb)
    hmaxtau = np.max(captau)
    hmaxArea0 = np.max(capArea0)
    
    
    if AllSigs:
        for i in range(len(GDs)-1):
            for j in range(i+1,len(GDs)):

                fullsteptdeb = plotSig(ax4,hmaxtdeb,steptdeb,fullsteptdeb,tdebs[i],tdebs[j],i,j)

                fullsteptau = plotSig(ax5,hmaxtau,steptau,fullsteptau,taus[i],taus[j],i,j)
                
                fullstepArea0 = plotSig(ax6,hmaxArea0,stepArea0,fullstepArea0,Area0[i],Area0[j],i,j)
    else:
        for i,j in sigpairs:
            
            fullsteptdeb = plotSig(ax4,hmaxtdeb,steptdeb,fullsteptdeb,tdebs[i],tdebs[j],i,j)

            fullsteptau = plotSig(ax5,hmaxtau,steptau,fullsteptau,taus[i],taus[j],i,j)
                
            fullstepArea0 = plotSig(ax6,hmaxArea0,stepArea0,fullstepArea0,Area0[i],Area0[j],i,j)

           
    plt.figure(fig4.number)
    fig4.savefig(P + '\\Figures\\AreaGrowth\\' + Title + '_Tstart.png')
    if not showbox:
        plt.close(fig4)

    plt.figure(fig5.number)
    fig5.savefig(P + '\\Figures\\AreaGrowth\\'+ Title +  '_TauGrowth.png')
    if not showbox:
        plt.close(fig5)

    plt.figure(fig6.number)
    fig6.savefig(P + '\\Figures\\AreaGrowth\\'+ Title +  '_StartingArea.png')
    if not showbox:
        plt.close(fig6)
 
    return    


print('Function defined')

#### Compute non-parametric significativity

In [ ]:
def plotSig(ax,hmax,step,fullstep,data1,data2,pos1,pos2):
    
    s,p = ranksums(data1,data2)
                        
    if pos2 == pos1+1: 
        h = np.median([np.median(data1),np.median(data2)])
        ax.plot([pos1+0.2, pos2-0.2], [h ,h], 'w-',zorder=0)
        ax.text((pos1+pos2)/2,h+0.2*step,'p = ' + str(round(p*1000)/1000), ha='center',fontsize='small')
        ax.set_ylim(top=hmax+fullstep+step)
            
    else:       
        h = hmax
        fullstep += step
        ax.plot([pos1, pos2], [h+fullstep ,h+fullstep], 'w-',zorder=0)
        ax.text((pos1+pos2)/2,h+fullstep+0.2*step,'p = ' + str(round(p*1000)/1000), ha='center',fontsize='small')
        ax.set_ylim(top=h+fullstep+step)

    return(fullstep)


### Compute correlations with size

In [ ]:
def SizeCorr(GDs,labels, **kwargs):
    
    corrmethod = 'pearson'
    dfcols = ['A0fit','Area','Tau','tdeb']
    colslab = dfcols
    colors = mtl.repmat([0.8, 0, 0.7],len(GDs),1)

    
    for key, value in kwargs.items(): 
        if key == 'corrmethod':
            corrmethod = value 
        elif key == 'columns':
            dfcols = value
        elif key == 'columnslabels':
            colslab = value
        elif key == 'colors':
            colors = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')

        
    for GD,lab,colo in zip(GDs,labels,colors) :
        
        GDtoCorr = GD.loc[GD['Img'] == 0, dfcols]
        corrMat = GDtoCorr.corr(method=corrmethod)
        
        plt.figure(dpi=250)
        plt.title('Correlation heatmap for ' + lab)
        sns.heatmap(corrMat,cmap = 'YlGnBu',linewidths=.5,square=True,vmin=-1,vmax=1,annot=True,fmt=".3f")
        
        for i in range(1,len(dfcols)):
            
            x,y = (GDtoCorr[dfcols[0]],GDtoCorr[dfcols[i]])
            
            linreg = linregress(x,y)
            
            
            fig,ax = plt.subplots(dpi=200)
            ax.set_title('Correlation between ' + dfcols[0] + ' and ' + dfcols[i] +'.\n Experiment : ' + lab)
            ax.plot(x,linreg.intercept + linreg.slope*x,'-w',lw = 1, label=f"Slope = {linreg.slope:.2f}")
            ax.plot(x,y,'o',color = colo,
                    label =f"CorrCoeff = {linreg.rvalue:.3f}\nP-value : {linreg.pvalue:.3f}")
            ax.set_xlabel(colslab[0])
            ax.set_ylabel(colslab[i])
            ax.legend()
            
        
    

## Enter data and run

### Data details

In [ ]:
Scale210903 = 1.94 # Spatial scale (µm/px) for 16X 03-09-2021
FPH210903 = 2 # Frames per hour

Scale210914 = 1.24 # Spatial scale (µm/px) for 25X 14-09-2021
FPH210914 = 2 # Frames per hour

Scale210927 = 1.24 # Spatial scale (µm/px) for 25X 27-09-2021
FPH210927 = 2 # Frames per hour

Scale211022 = 0.97 # Spatial scale (µm/px) for 32X 27-09-2021
FPH211022 = 2 # Frames per hour

Scale211105 = 1.55 # Spatial scale (µm/px) for 20X 05-11-2021
FPH211105 = 2 # Frames per hour


Scale220107 = 1.94 # Spatial scale (µm/px) for 16X 07-01-2022
FPH220107 = 2 # Frames per hour

Scale220124 = 1.94 # Spatial scale (µm/px) for 16X 22-01-2022
FPH220124 = 2 # Frames per hour

# Paths to data
P210903_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\1mlh'
P210903_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\500ulh'
P210914_Ct = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Ctrl'
P210914_M125 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Manitol125'
P210927_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\1mlh'
P210927_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\500ulh'
P211022_Ct1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl1'
P211022_Ct2 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl2'
P211105_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\BSA'
P211105_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\NoCoat'
P211222_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\BSA'
P211222_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\NoCoat'
P220107_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade3'
P220107_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade1'
P220112_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade3'
P220112_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade1'
P220114_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade3'
P220114_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade1'
P220124_Ct1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220124_DV6_Ctrls\Ctrl1'
P220124_Ct2 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220124_DV6_Ctrls\Ctrl2'

# Path for figures
PFig = r'd:\Users\laplaud\Desktop\PostDoc\Data\Microflu'


print('Data choice made.')


### Colors definition

In [ ]:
# Colors

LightGreen = [0.5, 1, 0.3]
Green = [0.3, 0.7, 0.2]
DarkGreen = [0.1, 0.4, 0.1]
PastelGreen = [0.5, 1, 0.8]
DarkPastelGreen = [0.2, 0.6, 0.5]

Pink = [1, 0.3, 0.5]
Red = [0.8, 0.2, 0.2]
DarkRed = [0.6, 0.1, 0.2]

LightBlue = [0.3, 0.5, 1]
Blue = [0.2, 0.3, 0.7]
DarkBlue = [0.1, 0.1, 0.4]

DarkPurple = [0.4, 0, 0.3]
Purple = [0.55, 0, 0.45]
LightPurple = [0.8, 0, 0.7]

Yellow = [0.8, 0.8, 0.2]
DarkYellow = [0.5, 0.5, 0.2]


### Load area fit data

In [ ]:
## Expe 210903
GlobalData210903_5 = pd.read_csv(P210903_5 + '\\GlobalData210903_500ulh_AreaContFit.csv', index_col = 'Ind')   

## Expe 210903
GlobalData210903_1 = pd.read_csv(P210903_1 + '\\GlobalData210903_1mlh_AreaContFit.csv', index_col = 'Ind')   

## Expe 210914 ctrl
GlobalData210914_Ct = pd.read_csv(P210914_Ct + '\\GlobalData210914_Ct_AreaContFit.csv', index_col = 'Ind')    

## Expe 210914 manitol 125mM
GlobalData210914_M125 = pd.read_csv(P210914_M125 + '\\GlobalData210914_M125_AreaContFit.csv', index_col = 'Ind')    

## Expe 210927
GlobalData210927_5 = pd.read_csv(P210927_5 + '\\GlobalData210927_5_AreaContFit.csv', index_col = 'Ind')    
GlobalData210927_1 = pd.read_csv(P210927_1 + '\\GlobalData210927_1_AreaContFit.csv', index_col = 'Ind')    

## Expe 211022
GlobalData211022_Ct2 = pd.read_csv(P211022_Ct2 + '\\GlobalData211022_Ct2_AreaContFit.csv', index_col = 'Ind')    
GlobalData211022_Ct1 = pd.read_csv(P211022_Ct1 + '\\GlobalData211022_Ct1_AreaContFit.csv', index_col = 'Ind')    

## Expe 211105
GlobalData211105_Degas = pd.read_csv(P211105_Degas + '\\GlobalData211105_Degas_AreaContFit.csv', index_col = 'Ind')    
GlobalData211105_Deg_Bsa = pd.read_csv(P211105_Deg_Bsa + '\\GlobalData211105_Deg_Bsa_AreaContFit.csv', index_col = 'Ind')    

## Expe 211222
GlobalData211222_Degas = pd.read_csv(P211222_Degas + '\\GlobalData211222_Degas_AreaContFit.csv', index_col = 'Ind')    
GlobalData211222_Deg_Bsa = pd.read_csv(P211222_Deg_Bsa + '\\GlobalData211222_Deg_Bsa_AreaContFit.csv', index_col = 'Ind')    

## Expe 220107
GlobalData220107_S1 = pd.read_csv(P220107_S1 + '\\GlobalData220107_S1_AreaContFit.csv', index_col = 'Ind')    
GlobalData220107_S3 = pd.read_csv(P220107_S3 + '\\GlobalData220107_S3_AreaContFit.csv', index_col = 'Ind')   

## Expe 220112
GlobalData220112_S1 = pd.read_csv(P220112_S1 + '\\GlobalData220112_S1_AreaContFit.csv', index_col = 'Ind')    
GlobalData220112_S3 = pd.read_csv(P220112_S3 + '\\GlobalData220112_S3_AreaContFit.csv', index_col = 'Ind')  

## Expe 220112
GlobalData220114_S1 = pd.read_csv(P220114_S1 + '\\GlobalData220114_S1_AreaContFit.csv', index_col = 'Ind')    
GlobalData220114_S3 = pd.read_csv(P220114_S3 + '\\GlobalData220114_S3_AreaContFit.csv', index_col = 'Ind')  

## Expe 220124
GlobalData220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_AreaContFit.csv', index_col = 'Ind')    
GlobalData220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_AreaContFit.csv', index_col = 'Ind')   

print('Loading OK.')

In [ ]:
print(GlobalData220124_Ct1)

### Plots by experiments

#### Ctrls V5

In [ ]:
# No degas flux controls
# GlobalData_5 = GlobalData210927_5.append(GlobalData210903_5)
# GlobalData_1 = GlobalData210927_1.append(GlobalData210903_1)

# compareAreas([GlobalData_5,GlobalData_1],['500 µl/h','1 ml/h'],
#           [Green, DarkGreen], PFig, 'V5_FluxControls_Pooled', showcurve = True, showbox = True);

# ## Ctrls gas & degas
# compareAreas([GlobalData211022_Ct1,GlobalData211022_Ct2],['Chamber 1','Chamber 2'],
#          [Red, DarkRed], PFig, 'V5_Ctrls', showcurve = True, showbox = True);

# compareAreas([GlobalData211105_Degas,GlobalData211222_Degas],['Degas0511','Degas2212'],
#          [Blue, DarkBlue], PFig, 'V5_Degas_2days', showcurve = True, showbox = True);

# GlobalDataCtrl = GlobalData211022_Ct1.append(GlobalData211022_Ct2)
# GlobalDataDegased = GlobalData211222_Degas.append(GlobalData211105_Degas)

# compareAreas([GlobalDataCtrl,GlobalDataDegased],['Ctrl','Degased'],
#            [Red, Blue], PFig, 'V5_DegasControls_Pooled', showcurve = True, showbox = True);



#### Stade comparison DV6

In [ ]:
GlobalData_S1 = GlobalData220107_S1.append(GlobalData220112_S1)
GlobalData_S3 = GlobalData220107_S3.append(GlobalData220112_S3)

GlobalData_S1 = GlobalData_S1.append(GlobalData220114_S1)
GlobalData_S3 = GlobalData_S3.append(GlobalData220114_S3)

# SizeCorr([GlobalData_S1,GlobalData_S3],['Stade1_Pooled','Stade3_Pooled'],colors = [Yellow, DarkYellow],
#          columns = ['A0fit','Area','Tau','tdeb'],
#          columnslabels = ['A0fit (mm²)', 'A0exp (mm²)', 'Tau (min)', 'Tstart (min)'])



SizeCorr([GlobalData220107_S1, GlobalData220107_S3],
              ['Stade 1 - 0701','Stade 3 - 0701'],colors= [LightGreen, Green],
         columns = ['A0fit','Area','Tau','tdeb','fitR2'],
         columnslabels = ['A0fit (mm²)', 'A0exp (mm²)', 'Tau (min)', 'Tstart (min)','R2 fit'])

SizeCorr([GlobalData220112_S1, GlobalData220112_S3],
              ['Stade 1 - 1201','Stade 3 - 1201'],colors= [LightBlue, Blue],
         columns = ['A0fit','Area','Tau','tdeb','fitR2'],
         columnslabels = ['A0fit (mm²)', 'A0exp (mm²)', 'Tau (min)', 'Tstart (min)','R2 fit'])

SizeCorr([GlobalData220114_S1, GlobalData220114_S3],
              ['Stade 1 - 1401','Stade 3 - 1401'],colors= [Pink, Red],
         columns = ['A0fit','Area','Tau','tdeb','fitR2'],
         columnslabels = ['A0fit (mm²)', 'A0exp (mm²)', 'Tau (min)', 'Tstart (min)','R2 fit'])


# compareAreas([GlobalData220107_S1, GlobalData220107_S3],
#              ['Stade 1','Stade 3'],[LightGreen, Green], 
#              PFig, 'DV6_Stades_0701', showcurve = True, showbox = True);

# compareAreas([GlobalData220112_S1, GlobalData220112_S3],
#              ['Stade 1','Stade 3'],[LightBlue, Blue], 
#              PFig, 'DV6_Stades_1201', showcurve = True, showbox = True);

# compareAreas([GlobalData220114_S1, GlobalData220114_S3],
#              ['Stade 1','Stade 3'],[Pink, Red], 
#              PFig, 'DV6_Stades_1401', showcurve = True, showbox = True, indiplots = True);

# compareAreas([GlobalData220107_S1, GlobalData220112_S1, GlobalData220114_S1,],
#              ['S1_0701','S1_1201','S1_1401'],[LightGreen, LightBlue, Pink], 
#              PFig, 'DV6_S1_All', showcurve = True, showbox = True);


# compareAreas([GlobalData220107_S3,GlobalData220112_S3,GlobalData220114_S3],
#              ['S3_0701','S3_1201','S3_1401'],[Green, Blue, Red], 
#              PFig, 'DV6_S3_All', showcurve = True, showbox = True);


# compareAreas([GlobalData220107_S1, GlobalData220107_S3, GlobalData220112_S1,GlobalData220112_S3, GlobalData220114_S1,GlobalData220114_S3],
#              ['S1_0701','S3_0701','S1_1201','S3_1201','S1_1401','S3_1401'],[LightGreen, Green, LightBlue, Blue, Pink, Red], 
#              PFig, 'DV6_Stades_All', showcurve = True, showbox = True);

# compareAreas([GlobalData_S1, GlobalData_S3],
#              ['Stade 1','Stade 3'],[Yellow, DarkYellow], 
#              PFig, 'DV6_Stades_Pooled', showcurve = True, showbox = True);



#### Hyperosmotic growth V5

In [ ]:
# compareAreas([GlobalData210914_Ct,GlobalData210914_M125],['Ctrl','Manitol'],
#             [PastelGreen, DarkPastelGreen], PFig, 'V5_Manitol125', showcurve = True, showbox = True);

#### Ctrls DV6 degas Stade1

In [ ]:
# compareAreas([GlobalData220124_Ct1,GlobalData220124_Ct2],['Ctrl1','Ctrl2'],
#             [PastelGreen, DarkPastelGreen], PFig, 'DV6_Degas_S1_Ctrls', showcurve = True, showbox = True);

## Test zone
